
# Segmentação de Clientes

##### A segmentação de clientes é o processo de dividir os clientes em grupos com base em características comuns, para que as empresas possam comercializar para cada grupo de forma eficaz e adequada ou compreender o padrão de consumo dos clientes.

In [ ]:
# Imports

# Manipulação e visualização de dados
import time
import sklearn
import datetime
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib as m
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

# Formatação dos gráficos
plt.style.use('fivethirtyeight')
plt.figure(1 , figsize = (15 , 6))
%matplotlib inline

# Ignora warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Para instalar uma versão específica do Scikit-Learn, por exemplo:
!pip install -q -U scikit-learn==0.23.1

In [ ]:
# Carrega o dataset
df = pd.read_csv("dados/dataset.csv", encoding = 'utf-8')

In [ ]:
# Shape
df.shape

In [ ]:
# Visualiza os dados
df.head(5)

### Análise Exploratória

Vamos explorar os dados por diferentes perspectivas e compreender um pouco mais o relacionamento entre as variáveis.

In [ ]:
# Verifica o total de valores únicos por coluna
df.nunique()

In [ ]:

df.dtypes

In [ ]:
# Resumo das colunas numéricas
df.describe()

In [ ]:
# Lista para receber o total de pedidos
total_pedidos = []

Loop para criar a tabela pivot totalizando os itens por transação.

In [ ]:
%%time

# P cada id e cada grupo do 'group by' por id_transacao
for k, group in df.groupby('id_transacao'):
    
    # Para cada id e cada grupo do group by por horario_pedido
    for m, n in group.groupby('horario_pedido'):
        
        # Para cada item de cada grupo
        id_transacao = k
        horario_pedido = m
        localidade = n['localidade'].values.tolist()[0]
        bebida = 0
        pizza = 0
        sobremesa = 0
        salada = 0
        n = n.reset_index(drop = True)
        
        # conta itens pedidos
        for i in range(len(n)):
            item = n.loc[i, 'nome_item']
            num = n.loc[i, 'quantidade_item']
            
            if item == 'bebida':
                bebida = bebida + num
            
            elif item == 'pizza':
                pizza = pizza + num
            
            elif item == 'sobremesa':
                sobremesa = sobremesa + num
            
            elif item == 'salada':
                salada = salada + num
        
        output = [id_transacao, horario_pedido, localidade, bebida, pizza, sobremesa, salada]
        total_pedidos.append(output)
        

In [ ]:
# list to dataframe
df_item_pedidos = pd.DataFrame(total_pedidos)

In [ ]:
# Add nomes de colunas
df_item_pedidos.columns = ['id_transacao', 'horario_pedido', 'localidade', 'bebida', 'pizza', 'sobremesa', 'salada']

In [ ]:

df_item_pedidos.shape

In [ ]:

df_item_pedidos.nunique()

In [ ]:
df.head(5) # dados originais

In [ ]:
df_item_pedidos.head(5)

Com a tabela criada pivot, temos uma nova perspectiva dos dados.

### Ajuste de Índices

PAra segmentar corretamente os pedidos precisa identificar uma coluna de id para cada registro, logo nao pode-se usar id_transacao pois esta coluna representa um dado valido e nao unico.

In [ ]:
# Dataset
df_item_pedidos

In [ ]:
# Índice
df_item_pedidos.index

In [ ]:
# Fazemos o reset no índice e gravamos o resultado em outro dataframe
df_item_pedidos_idx = df_item_pedidos.reset_index()

In [ ]:
# Pronto, agora temos uma coluna de ID com valor único para cada registro
df_item_pedidos_idx.head()

In [ ]:
# Dataset
df_item_pedidos.head(3)

### Análise Descritiva

### Distplot dos Atributos Usados Para Segmentação

In [ ]:
# Plot

plt.figure(1 , figsize = (19 , 6))

n = 0 

# Percorre colunas
for x in ['pizza' , 'sobremesa' , 'salada', 'bebida', 'localidade']:
    n += 1
    plt.subplot(1 , 5 , n)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
    sns.distplot(df_item_pedidos[x] , bins = 20)
    plt.title('Distplot de {}'.format(x))
plt.show()

### Gráfico de Total de Pedidos Por Localidade

In [ ]:
# Plot
plt.figure(1 , figsize = (15 , 5))
sns.countplot(y = 'localidade' , data = df_item_pedidos)
plt.show()

### Regplot dos Atributos Usados Para Segmentação

In [ ]:
# Relação Entre os Atributos

plt.figure(1 , figsize = (15 , 7))

n = 0 

# percorre atributos
for x in ['pizza' , 'sobremesa' , 'salada', 'bebida']:
    for y in ['pizza' , 'sobremesa' , 'salada', 'bebida']:
        n += 1
        plt.subplot(4 , 4 , n)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
        sns.regplot(x = x , y = y , data = df_item_pedidos)
        plt.ylabel(y)
plt.show()

### Definindo as Variáveis Para Segmentação

Remove id_transacao, horario_pedido, localidade para clusterização.

In [ ]:
# Filtrando o dataframe por colunas 
df_item_pedidos_idx[['index', 'bebida', 'pizza', 'sobremesa', 'salada']]

In [ ]:
# Novo DF
df_novo = df_item_pedidos_idx[['index', 'bebida', 'pizza', 'sobremesa', 'salada']]

In [ ]:
# Dataset
df_novo.head()

In [ ]:
# com duas var
X1 = df_novo[['pizza' , 'sobremesa']].iloc[: , :].values

In [ ]:
# Lista do WCSS - kmeans
wcss_X1 = []

#### Segmentação 1 - Encontrando o Valor Ideal de Clusters

Testando diferentes valores de K (valores de cluster) entre 2 e 10.

Para a inicialização dos clusters, o algoritmo k-means++ oferece convergência rápida para resultado final.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
# Loop para testar os valores de K
for n in range(2, 11):
    modelo = (KMeans(n_clusters = n,
                     init = 'k-means++', 
                     n_init = 10,
                     max_iter = 300, 
                     tol = 0.0001,  
                     random_state = 111, 
                     algorithm = 'elkan'))
    modelo.fit(X1)
    wcss_X1.append(modelo.inertia_)

In [ ]:
# Plot
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(2 , 11) , wcss_X1 , 'o')
plt.plot(np.arange(2 , 11) , wcss_X1 , '-' , alpha = 0.5)
plt.xlabel('Número de Clusters') , plt.ylabel('WCSS')
plt.show()

Acima a curva ELBOW. Valor de 'n' deve ser alinhado com necessidade do negocio (x propagandas de marketing por ex). Aqui vamos por 2 e ver os resuiltados

In [ ]:
# Criação do modelo
modelo_seg1 = KMeans(n_clusters = 2, 
                     init = 'k-means++', 
                     n_init = 10, 
                     max_iter = 300, 
                     tol = 0.0001,  
                     random_state = 111, 
                     algorithm = 'elkan')

In [ ]:
# Treinamento
modelo_seg1.fit(X1)

In [ ]:
# Extração dos labels
labels1 = modelo_seg1.labels_
labels1

In [ ]:
# Extração dos centróides
centroids1 = modelo_seg1.cluster_centers_
centroids1

#### Segmentação 1 - Visualização e Interpretação dos Segmentos

In [ ]:
# Plot

# Parâmetros do Meshgrid
h = 0.02
x_min, x_max = X1[:, 0].min() - 1, X1[:, 0].max() + 1
y_min, y_max = X1[:, 1].min() - 1, X1[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = modelo_seg1.predict(np.c_[xx.ravel(), yy.ravel()]) 
plt.figure(1 , figsize = (15, 7) )
plt.clf()
Z = Z.reshape(xx.shape)

# Plotando
plt.imshow(Z, 
           interpolation = 'nearest', 
           extent = (xx.min(), xx.max(), yy.min(), yy.max()),
           cmap = plt.cm.Set2, 
           aspect = 'auto', 
           origin = 'lower')

# Plotando pontos
plt.scatter( x = 'pizza', y = 'sobremesa', data = df_novo, c = labels1, s = 200 )
plt.scatter(x = centroids1[: , 0], y =  centroids1[: , 1], s = 300, c = 'red', alpha = 0.5)
plt.xlabel('Pizza')
plt.ylabel('Sobremesa')
plt.show()

Vemos que o ponto VERMELHO é o centroide de cada cluster(segmento), na area em verde os clientes pediram 0,1,2 pizzas e houve sobremesa.

Na área cinza, pediram mais pizza e com o aumento houve mais pedidos de sobremesas

Então no cluster 1 - Clientes que pedem menos Pizzas. Todos pedem sobremesa.
E no Cluster 2 - Clientes que pedem mais Pizzas. Todos pedem sobremesa em volume maior.

A loja poderia oferecer sobremesa gratis caso compre mais pizzas de maior valor.